In [2]:
import pandas as pd

In [3]:
df_regiotabel = pd.read_excel('awbzregiotabel.xlsx')
df_regiotabel = df_regiotabel[['Postcode', 'Gemeentecode']]
df_regiotabel['Gemeentecode'] = 'GM' + df_regiotabel['Gemeentecode'].astype(str).str.zfill(4)
df_regiotabel.set_index('Gemeentecode', inplace=True)
df_regiotabel.head(2)

,Postcode
Gemeentecode,
GM0363,1000
GM0363,1001


In [4]:
df_zkhregios = pd.read_excel('zkh_regios.xlsx')
df_zkhregios.set_index('Codes en namen van gemeenten/Code (code)', inplace=True)
df_zkhregios.index = df_zkhregios.index.str.strip()
df_zkhregios.head(2)

,Regio's,Codes en namen van gemeenten/Naam (naam),Lokaliseringen van gemeenten/COROP-gebieden/Code (code),Lokaliseringen van gemeenten/COROP-gebieden/Naam (naam),Lokaliseringen van gemeenten/GGD-regio's/Code (code),Lokaliseringen van gemeenten/GGD-regio's/Naam (naam),Lokaliseringen van gemeenten/Jeugdregio's/Code (code),Lokaliseringen van gemeenten/Jeugdregio's/Naam (naam),Lokaliseringen van gemeenten/Provincies/Code (code),Lokaliseringen van gemeenten/Provincies/Naam (naam),...,Lokaliseringen van gemeenten/Toeristengebieden/Code (code),Lokaliseringen van gemeenten/Toeristengebieden/Naam (naam),Lokaliseringen van gemeenten/Veiligheidsregio's/Code (code),Lokaliseringen van gemeenten/Veiligheidsregio's/Naam (naam),Lokaliseringen van gemeenten/Zorgkantoorregio's/Code (code),Lokaliseringen van gemeenten/Zorgkantoorregio's/Naam (naam),Lokaliseringen van gemeenten/ROAZregio's/Code (code),Lokaliseringen van gemeenten/ROAZregio's/Naam (naam),Lokaliseringen van gemeenten/RAVregio's/Code (code),Lokaliseringen van gemeenten/RAVregio's/Naam (naam)
Codes en namen van gemeenten/Code (code),,,,,,,,,,,,,,,,,,,,,
GM1680,Aa en Hunze,Aa en Hunze ...,CR07,Noord-Drenthe ...,GG0706,GGD Drenthe ...,JZ04,Drenthe ...,PV22,Drenthe ...,...,TR12,"Groningse, Friese en Drentse Zandgronden ...",VR03,Drenthe ...,ZK06,Drenthe ...,ROAZ01,Acute Zorg Noord Nederland,VR03,Drenthe ...
GM0358,Aalsmeer,Aalsmeer ...,CR23,Groot-Amsterdam ...,GG3406,GGD Amsterdam ...,JZ13,Amsterdam-Amstelland ...,PV27,Noord-Holland ...,...,TR06,Hollands-Utrechtse meren ...,VR13,Amsterdam-Amstelland ...,ZK01,Amstelland en De Meerlanden ...,ROAZ02,Netwerk Acute Zorg Noordwest,VR13,Amsterdam-Amstelland ...


In [42]:

df = df_regiotabel.join(df_zkhregios) # , how='outer')
columns = {
   "Lokaliseringen van gemeenten/Veiligheidsregio's/Code (code)": 'VEILIGHEIDSREGIO',
   "Lokaliseringen van gemeenten/COROP-gebieden/Code (code)": 'COROP',
   "Lokaliseringen van gemeenten/GGD-regio's/Code (code)": 'GGD',
   "Lokaliseringen van gemeenten/Jeugdregio's/Code (code)": 'JEUGD',
   "Lokaliseringen van gemeenten/Provincies/Code (code)": "PROVINCIE", 
   "Lokaliseringen van gemeenten/Regionale Energiestrategie regio’s/Code (code)": 'ENERGIE',
   "Lokaliseringen van gemeenten/Toeristengebieden/Code (code)": 'TOERIST',
   "Lokaliseringen van gemeenten/Zorgkantoorregio's/Code (code)": 'ZORGKANTOOR',
   "Lokaliseringen van gemeenten/ROAZregio's/Code (code)": 'ROAZ',
   "Lokaliseringen van gemeenten/RAVregio's/Code (code)": 'RAV',

}

df = df[['Postcode', *columns]].rename(columns=columns)

df.set_index('Postcode', inplace=True)

for column in df:  # remove whitespaces
    df.loc[:, column] = df[column].str.replace(" ","")

root = {}
root['postcodes'] = {index: row.to_dict() for index, row in df.iterrows() if not row.isna().any()}

In [43]:
import json
with open('regios.json', 'w') as filepath:
    json.dump(root, filepath)

Vind plaasten van ziekenhuizen json

In [6]:
import json

In [7]:
json_file_path = "ziekenhuizen.json"

with open(json_file_path, 'r') as j:
     root = json.loads(j.read())
root.keys()
zkh_postcodes = [int(feature['properties']['pc'][:4]) for feature in root['features']]

In [8]:
for zkh_postcode in zkh_postcodes:
    if zkh_postcode not in df.index:
        print(zkh_postcode)  # should return nothing